# Notebook for running bwa-mem on pronghorn

This notebook will run through everyting from bwa-mem, sam2bam, and sorting  
Currently set up to submit one slurm script per fastq for bwa-mem. Not for sam2bam though

In [1]:
### This is just the list of packages I import each time. Not necessary whatsoever. Really only use the first 8 or so.  

import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

### Orginization of directories

There's only a few things you need to set and everything else should be fairly automated. This root directory is the main one. 

In [2]:
root = "/data/gpfs/assoc/denovo/tfaske/rabbit/full"

In [3]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbit/full


In [4]:
pwd

'/data/gpfs/assoc/denovo/tfaske/rabbit/full'

# Actual Mapping 
 
 
 First thing below is finding all the fastq.gz files. Must change to directory and proper ending for it to work

In [5]:
fastq_files = []
os.chdir('{}/{}'.format(root, 'demult/fastq/')) ### fastq files in this directory
files = !find . -name '*fastq.gz' #make sure you change endings
files = [os.path.abspath(x) for x in files]
for x in files:
    fastq_files.append(x)
fastq_files = sorted(fastq_files)

In [6]:
len(fastq_files),fastq_files[0]

(608,
 '/data/gpfs/assoc/denovo/tfaske/rabbit/full/demult/fastq/EN_AH_1.fastq.gz')

In [68]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbit/full


In [8]:
#Point to indexed (through bwa) reference assembly. This index was done by dDocent. 

assembly = "/data/gpfs/assoc/denovo/tfaske/rabbit/full/assembly/reference.fasta"

#### Function for defining bwa-mem calls with a few notes

Need to change ID split to parse out just the SP_POP_ID designation

In [9]:
# -k INT minimum seed length [19]
# -w INT band width for banded alignment [100]
# -r FLOAT look for internal seeds inside a seed longer than {-k} * FLOAT [1.5]
# -T INT minimum score to output [30]
# -R STR read group header line such as '@RG\tID:foo\tSM:bar' [null]

#@lview.remote()
def run_bwamem(args):
    import os, multiprocessing, socket
    cpus = 1
    assembly, fq, outdir = args
    ID = fq.split('/')[10] ### need to change this to match your ID 
    ID = ID.split('.fastq.')[0] ### This too 
    sam = os.path.join(outdir, "{}.sam".format(os.path.basename(ID)))
    cmd = r"bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\tID:%s\tLB:%s\tSM:%s\tPL:ILLUMINA' %s %s > %s" % (ID,ID,ID,assembly,fq,sam)
                                                               
    res = None
    res = cmd
    return  cmd 

# Write sh for each file and then bash to submit file

This will make directories within root and scripts are set up to move throughout

In [10]:
!mkdir SNPcall

mkdir: cannot create directory ‘SNPcall’: File exists


In [11]:
!mkdir SNPcall/bwa

In [12]:
!mkdir SNPcall/shbwa

In [13]:
bwa_dir = os.path.join(root,'SNPcall/bwa/')
assert(bwa_dir)

In [14]:
### creates a list of commands for bwa-mem for each fastq file

res = []
for f in fastq_files:
    r = run_bwamem((assembly, f, bwa_dir))
    res.append(r)

In [15]:
res[0]

"bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\\tID:EN_AH_1\\tLB:EN_AH_1\\tSM:EN_AH_1\\tPL:ILLUMINA' /data/gpfs/assoc/denovo/tfaske/rabbit/full/assembly/reference.fasta /data/gpfs/assoc/denovo/tfaske/rabbit/full/demult/fastq/EN_AH_1.fastq.gz > /data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa/EN_AH_1.sam"

In [16]:
pwd

'/data/gpfs/assoc/denovo/tfaske/rabbit/full'

In [18]:
fq_ID = [f.split('/')[10].split('.fastq.')[0] for f in fastq_files]
print(len(fq_ID),fq_ID[0])

608 EN_AH_1


#### Below selects options for slurm submission and is a function for creating a slurm script per fastq

In [19]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'bwa_rabfull'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [20]:
def write_bwamem_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,fq_ID,cmds):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    for i in range(0,len(cmds)):
        with open("SNPcall/shbwa/run_bwamem_%s.sh" % (fq_ID[i]), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_bwamem
#SBATCH --output output_bwamem_%s.txt
#SBATCH --mail-user=%s \n\n
    
%s \n""" % (account,partition,time,int(cpus),int(mem_cpu),fq_ID[i],fq_ID[i],email,cmds[i]))

In [21]:
write_bwamem_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,fq_ID,res)

#### finds all bwa slurm scripts and writes bash script to sbatch them

In [22]:
shbwa_files = []
os.chdir('{}/{}'.format(root, 'SNPcall/shbwa'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shbwa_files.append(x)
shbwa_files = sorted(shbwa_files)

In [23]:
len(shbwa_files),shbwa_files[0]

(608,
 '/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/shbwa/run_bwamem_EN_AH_1.sh')

In [24]:
def write_bash_bwamem_sh(sh_files):
    with open("SNPcall/run_bash_bwa.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [25]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbit/full


In [26]:
write_bash_bwamem_sh(shbwa_files)

# Run run_bash_bwamem_sh locally
    cd /data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall 
    source activate py36
    bash run_bash_bwa.sh

# Create a sam2bam slurm script

Currently creates one but could be expanded to run one per fastq like above 

In [31]:
fqs = !find demult/fastq/ -name "*fastq.gz"
sams = !find $bwa_dir -name "*.sam"

In [32]:
len(fqs), len(sams)

(608, 608)

In [33]:
print(fqs[0])
print(sams[0])

demult/fastq/EN_SL_2.fastq.gz
/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa/EN_HO_6.sam


#### just makes sure fastq files and sam files match

In [35]:
fq_ID = [f.split('/')[2].split('.fastq.')[0] for f in fqs]
sam_ID = [s.split('/')[10].split('.sam')[0] for s in sams]

In [36]:
print(fq_ID[0])
print(sam_ID[0])

EN_SL_2
EN_HO_6


In [37]:
fq_sam_diff = list(set(fq_ID).difference(sam_ID))
len(fq_sam_diff), len(fqs) - len(sams)

(0, 0)

In [38]:
fq_sam_diff = list(set(fq_ID).difference(sam_ID))
fq_sam_diff

[]

### sam2bam for each file

In [58]:
def run_sam2bam(args):
    s,cpus = args
    bam = s.replace(".sam", ".bam")
    bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
    cmd = "samtools view -b %s -o %s\nsamtools sort -@ %s %s -o %s\nsamtools index %s\n" % (s,bam,cpus,bam,bam_sorted,bam_sorted)                                                      
    res = None
    res = cmd
    return  cmd 

In [59]:
### creates a list of commands for sam2bam for each fastq file
res_s2b = []
cpus = 1
for sam in sams:
    r = run_sam2bam((sam,cpus))
    res_s2b.append(r)

In [60]:
len(res_s2b), res_s2b[0]

(608,
 'samtools view -b /data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa/EN_HO_6.sam -o /data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa/EN_HO_6.bam\nsamtools sort -@ 1 /data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa/EN_HO_6.bam -o /data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa/EN_HO_6_sorted.bam\nsamtools index /data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa/EN_HO_6_sorted.bam\n')

#### Below selects options for slurm submission and is a function for creating a slurm script for sam2bam

In [48]:
!mkdir SNPcall/shsam2bam

In [64]:
sam_ID = [s.split('/')[10].split('.sam')[0] for s in sams]
sam_ID[0]

'EN_HO_6'

In [65]:
### select options
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'sam2bam'
time = '1-00:00:00' #time limit 1 days
cpus = 1
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [69]:
def write_sam2bam_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,sam_ID,cmds):
    for i in range(0,len(cmds)):
        with open("SNPcall/shsam2bam/run_sam2bam_%s.sh" % (sam_ID[i]), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_sam2bam
#SBATCH --output output_sam2bam_%s.txt
#SBATCH --mail-user=%s \n\n
    
%s \n""" % (account,partition,time,int(cpus),int(mem_cpu),sam_ID[i],sam_ID[i],email,cmds[i]))

In [70]:
write_sam2bam_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,sam_ID,res_s2b)

#### finds all sam2bam slurm scripts and writes bash script to sbatch them

In [74]:
shsam2bam_files = []
os.chdir('{}/{}'.format(root, 'SNPcall/shsam2bam'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shsam2bam_files.append(x)
shsam2bam_files = sorted(shsam2bam_files)

In [75]:
len(shsam2bam_files),shsam2bam_files[0]

(608,
 '/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/shsam2bam/run_sam2bam_EN_AH_1.sh')

In [76]:
def write_bash_sam2bam_sh(sh_files):
    with open("SNPcall/run_bash_sam2bam.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [77]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbit/full


In [78]:
write_bash_sam2bam_sh(shsam2bam_files)

# run_bash_sam2bam.sh
    cd /data/gpfs/home/tfaske/d/rabbit/full/SNPcall 
    source activate py36
    sbatch run_bash_sam2bam.sh


### Count sorted bam to keep an eye on progress

In [79]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa


In [81]:
sorted_bams = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    sorted_bams.append(x)
sorted_bams = sorted(sorted_bams)

#assert len(sorted_bams) == len(fastq_files)

len(sorted_bams), len(fqs), len(sams)

(608, 608, 608)

#### remove sams

In [83]:
!rm *sam